In [1]:
import os
import numpy as np
import pandas as pd
from usearch.io import load_matrix
from usearch.index import Index
from encode import vectorize_e5, vectorize_uform


/home/user1/miniconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 7 files: 100%|██████████| 7/7 [00:00<00:00, 96262.71it/s]


In [2]:
def load_vectors():
    vectors = load_matrix("./data/arxiv/abstract.e5-base-v2.fbin")
    return vectors.astype(np.float16)

In [3]:
index_path = "./data_usearch/arxiv/abstracts.usearch"
index = Index(dtype="f16", metric="cos", ndim=768)
if os.path.exists(index_path):
    index.load(index_path)
else:
    index.add(None, load_vectors(), log=True)
    index.save(index_path)

In [4]:
index.hardware_acceleration

'auto'

In [5]:
abstracts_path = "./data/arxiv/title_abstract.tsv"
abstracts = pd.read_csv(abstracts_path, sep="\t")

In [12]:
abstracts

,title,abstract
0,Calculation of prompt diphoton production cros...,A fully differential calculation in perturbati...
1,Sparsity-certifying Graph Decompositions,"We describe a new algorithm, the $(k,\ell)$-pe..."
2,The evolution of the Earth-Moon system based o...,The evolution of Earth-Moon system is describe...
3,A determinant of Stirling cycle numbers counts...,We show that a determinant of Stirling cycle n...
4,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,In this paper we show how to compute the $\Lam...
...,...,...
2288295,On the origin of the irreversibility line in t...,We report on measurements of the angular depen...
2288296,Nonlinear Response of HTSC Thin Film Microwave...,The non-linear microwave surface impedance of ...
2288297,Critical State Flux Penetration and Linear Mic...,The vortex contribution to the dc field (H) de...
2288298,Density of States and NMR Relaxation Rate in A...,We show that the density of states in an aniso...


In [13]:
abstracts.to_parquet('data_usearch/arxiv/abstracts.parquet')

/home/user1/miniconda3/envs/py310/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [7]:
sample_abstract = abstracts["abstract"][2]
vectorize_e5(sample_abstract)

array([[-3.7012e-01, -3.5572e-03, -4.3506e-01, -9.7229e-02,  1.0425e-01,
        -5.9082e-01, -8.3008e-02,  6.4648e-01, -7.9834e-02, -3.0908e-01,
         1.1102e-01,  6.9678e-01, -5.2344e-01,  6.2927e-02, -1.6687e-01,
         7.8662e-01,  2.4365e-01, -7.7637e-01,  6.8945e-01, -7.9932e-01,
        -2.2302e-01, -6.1182e-01,  6.5674e-01,  4.3530e-01, -2.0288e-01,
         2.4609e-01, -6.6528e-02,  5.6299e-01, -5.5273e-01, -7.6294e-02,
         2.2559e-01,  4.0210e-01,  5.5029e-01, -2.6758e-01, -4.7070e-01,
         1.5686e-01, -7.1240e-01, -1.2231e-01, -5.4980e-01,  1.9730e-02,
        -1.8225e-01, -3.5547e-01, -6.0645e-01,  6.2744e-01, -5.5176e-01,
        -8.4656e-02, -6.2158e-01,  8.5938e-01, -6.4258e-01, -5.9082e-01,
        -1.4595e-02,  9.2346e-02,  8.0273e-01,  3.7720e-02, -5.2979e-01,
         2.1509e-01, -3.5547e-01, -5.5859e-01, -1.5015e-01, -2.5415e-01,
         6.8408e-01,  9.9548e-02,  6.9618e-03, -4.9683e-01,  4.6844e-02,
         2.2595e-01,  9.1431e-02, -7.1533e-02, -2.1

In [8]:
vectorize_e5(sample_abstract).shape

(1, 768)

In [9]:
matches = index.search(vectorize_e5(sample_abstract), 10)
matches

usearch.Matches(10)

In [11]:
assert matches.keys[0] == 2